In [1]:
import re
import tarfile
import functools

import numpy as np

from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
import spacy as sp
nlp = sp.load('en')

In [3]:
def tokenize(sent):
    sent = nlp(sent)
    words = [token.orth_ for token in sent]
    pos = [token.pos_ for token in sent]
    
    return (words,pos)

In [4]:
def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true,
    only the sentences that support the answer are kept.
    '''
    data = []
    story = []
    storypos = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
            storypos = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q, qpos = tokenize(q)
            substory = None
            substorypos = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
                substorypos = [x for x in storypos if x]
            data.append((substory, substorypos, q, qpos, a))
            story.append('')
        else:
            sent, sentpos = tokenize(line)
            story.append(sent)
            storypos.append(sentpos)
    return data

In [5]:
def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file, retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: functools.reduce(lambda x, y: x + y, data)
    data = [(flatten(story), flatten(storypos), q, qpos, answer) 
            for story, storypos, q, qpos, answer in data if not max_length or len(flatten(story)) < max_length]
    return data

In [6]:
def vectorize_stories(data, word_idx, wordpos_idx, story_maxlen, query_maxlen):
    xs = []
    xspos = []
    xqs = []
    xqspos = []
    ys = []
    for story, storypos, query, querypos, answer in data:
        x = [word_idx[w] for w in story]
        xpos = [wordpos_idx[w] for w in storypos]
        xq = [word_idx[w] for w in query]
        xqpos = [wordpos_idx[w] for w in querypos]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        xs.append(x)
        xspos.append(xpos)
        xqs.append(xq)
        xqspos.append(xqpos)
        ys.append(y)
    print(x)
    return pad_sequences(xs, maxlen=story_maxlen), pad_sequences(xspos, maxlen=story_maxlen), pad_sequences(xqs, maxlen=query_maxlen),  pad_sequences(xqspos, maxlen=query_maxlen), np.array(ys)

In [27]:
RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 30
SENT_HIDDEN_SIZE = 100
QUERY_HIDDEN_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 40
print('RNN / Embed / Sent / Query = {}, {}, {}, {}'.format(RNN,
                                                           EMBED_HIDDEN_SIZE,
                                                           SENT_HIDDEN_SIZE,
                                                           QUERY_HIDDEN_SIZE))

RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 30, 100, 100


In [19]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)

In [20]:
# Default QA1 with 1000 samples
challenge = 'tasks_1-20_v1-2/en/qa1_single-supporting-fact_{}.txt'
# QA1 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt'
# QA2 with 1000 samples
#challenge = 'tasks_1-20_v1-2/en/qa2_two-supporting-facts_{}.txt'
# QA2 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt'
train = get_stories(tar.extractfile(challenge.format('train')))
test = get_stories(tar.extractfile(challenge.format('test')))

In [21]:
vocab = set()
vocabpos = set()
for story, storypos, q, qpos, answer in train + test:
    vocab |= set(story + q + [answer])
    vocabpos |= set(storypos + qpos)
vocab = sorted(vocab)
vocabpos = sorted(vocabpos)

In [22]:
# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
vocabpos_size = len(vocabpos) + 1
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
wordpos_idx = dict((c, i + 1) for i, c in enumerate(vocabpos))
story_maxlen = max(map(len, (x for x, _, _, _, _ in train + test)))
query_maxlen = max(map(len, (x for _, _, x, _, _ in train + test)))

In [23]:
x, xpos, xq, xqpos, y = vectorize_stories(train, word_idx, wordpos_idx, story_maxlen, query_maxlen)
tx, txpos, txq, txqpos, ty = vectorize_stories(test, word_idx, wordpos_idx, story_maxlen, query_maxlen)

[6, 20, 19, 18, 12, 1, 6, 21, 19, 18, 10, 1, 3, 16, 19, 18, 17, 1, 5, 21, 19, 18, 12, 1, 5, 14, 19, 18, 15, 1, 4, 21, 8, 19, 18, 10, 1, 3, 20, 19, 18, 15, 1, 6, 20, 19, 18, 15, 1, 6, 20, 19, 18, 12, 1, 3, 21, 19, 18, 11, 1]
[6, 16, 19, 18, 15, 1, 4, 20, 19, 18, 15, 1, 6, 16, 19, 18, 12, 1, 3, 14, 19, 18, 12, 1, 6, 20, 19, 18, 17, 1, 4, 16, 19, 18, 10, 1, 6, 21, 8, 19, 18, 11, 1, 6, 20, 19, 18, 9, 1, 6, 16, 19, 18, 17, 1, 3, 14, 19, 18, 11, 1]


In [24]:
print('vocab = {}'.format(vocab))
print('vocabpos = {}'.format(vocabpos))
print('x.shape = {}'.format(x.shape))
print('xpos.shape = {}'.format(xpos.shape))
print('xq.shape = {}'.format(xq.shape))
print('xqpos.shape = {}'.format(xqpos.shape))
print('y.shape = {}'.format(y.shape))
print('story_maxlen, query_maxlen = {}, {}'.format(story_maxlen, query_maxlen))

vocab = ['.', '?', 'Daniel', 'John', 'Mary', 'Sandra', 'Where', 'back', 'bathroom', 'bedroom', 'garden', 'hallway', 'is', 'journeyed', 'kitchen', 'moved', 'office', 'the', 'to', 'travelled', 'went']
vocabpos = ['ADP', 'ADV', 'DET', 'NOUN', 'PROPN', 'PUNCT', 'VERB']
x.shape = (1000, 66)
xpos.shape = (1000, 66)
xq.shape = (1000, 4)
xqpos.shape = (1000, 4)
y.shape = (1000, 22)
story_maxlen, query_maxlen = 66, 4


In [28]:
print('Build model...')

sentence = layers.Input(shape=(story_maxlen,), dtype='int32') #input the story as a sequence of tokens
encoded_sentence = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(sentence) #embed the tokens in the story
sentencepos = layers.Input(shape=(story_maxlen,), dtype='int32')
encoded_sentencepos = layers.Embedding(vocabpos_size, EMBED_HIDDEN_SIZE)(sentencepos)
merged_sentence = layers.concatenate([encoded_sentence, encoded_sentencepos])
merged_sentence = layers.Dropout(0.3)(merged_sentence)
encoded_sentence = RNN(EMBED_HIDDEN_SIZE)(merged_sentence)

question = layers.Input(shape=(query_maxlen,), dtype='int32')
encoded_question = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(question)
questionpos = layers.Input(shape=(query_maxlen,), dtype='int32')
encoded_qpos = layers.Embedding(vocabpos_size, EMBED_HIDDEN_SIZE)(questionpos)
merged_q = layers.concatenate([encoded_question, encoded_qpos])
merged_q = layers.Dropout(0.3)(merged_q)
encoded_question = RNN(EMBED_HIDDEN_SIZE)(merged_q)

merged = layers.concatenate([encoded_sentence, encoded_question])
merged = layers.Dropout(0.3)(merged)
preds = layers.Dense(vocab_size, activation='relu')(merged) 

model = Model([sentence, sentencepos, question, questionpos], preds)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Build model...


In [29]:
print('Training')
model.fit([x, xpos, xq, xqpos], y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.05)
loss, acc = model.evaluate([tx, txpos, txq, txqpos], ty,
                           batch_size=BATCH_SIZE)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

Training
Train on 950 samples, validate on 50 samples
Epoch 1/40
950/950 [==============================] - 11s - loss: 2.7258 - acc: 0.1642 - val_loss: 1.8003 - val_acc: 0.1200